In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/ETOHA

Mounted at /content/drive
/content/drive/MyDrive/ETOHA


In [2]:
# !pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.2 MB/s eta 0:00:00


In [3]:
behprepost_map = {
    0: "No",
    1: "Yes, pre-surgery only",
    2: "Yes, post-surgery only",
    3: "Yes, pre- and post-surgery",
    4: "Yes, no surgery"
}

diag_basis_map = {
    1: "Clinical examination only (history and physical exam)",
    2: "Clinical-diagnostic tests or exploratory surgery/autopsy (no microscopic confirmation)",
    4: "Specific biochemical and/or immunological laboratory tests",
    5: "Hematological or cytological confirmation (primary tumor or metastases); unclear if cytology or histology",
    6: "Histological confirmation of metastases only (including autopsy)",
    7: "Histological confirmation of the primary tumor (or unknown whether primary or metastases); possibly autopsy with histological confirmation"
}

diffgrad_map = {
    1: "Well differentiated (low-grade)",
    2: "Moderately differentiated (intermediate)",
    3: "Poorly differentiated (high-grade)",
    4: "Undifferentiated / anaplastic",
    9: "Unknown / not applicable / not determined"
}

behavior_map = {
    2: "In situ",
    3: "Malignant"
}

gender_map = {
    1: "Male",
    2: "Female"
}

her2_stat_map = {
    0: "0 (negative)",
    1: "1+ (negative)",
    2: "2+ (equivocal)",
    3: "3+ (positive)",
    4: "Not determined",
    7: "Not determined (7)",
    9: "Not assessable / unknown"
}

hr_stat_map = {
    0: "Negative",
    1: "Positive",
    9: "Not assessable / unknown"
}

mari_uitslag_map = {
    1: "MARI node negative",
    2: "ITC (≤ 0.2 mm)",
    3: "Micrometastasis (>0.2 - ≤2 mm)",
    4: "MARI node positive",
    5: "MARI node not removed",
    9: "Unknown outcome"
}

tumor_type_map = {
    501300: "Invasive breast carcinoma",
    502200: "Ductal carcinoma in situ",
    503200: "Lobular carcinoma in situ"
    # Add additional codes if you have them
}

neeja_map = {
    0: "No",
    1: "Yes"
}

neejaonb_map = {
    0: "No",
    1: "Yes",
    9: "Unknown"
}

swk_map = {
    0: "No",
    1: "Yes",
    8: "Not registered in this region"
}

vital_status_map = {
    0: "Alive",
    1: "Deceased"
}

later_map = {
    "1.0": "Left",
    "2.0": "Right",
    "X": "Unknown"
}

swk_uitslag_map = {
    1: "Negative sentinel node",
    2: "ITC (≤ 0.2 mm)",
    3: "Micrometastasis (>0.2 - ≤2 mm)",
    4: "Positive sentinel node (>2 mm)",
    9: "Sentinel node not found"
}

topo_sublok_map = {
    "C500": "Nipple/areola",
    "C501": "Central portion of breast",
    "C502": "Upper-inner quadrant",
    "C503": "Lower-inner quadrant",
    "C504": "Upper-outer quadrant",
    "C505": "Lower-outer quadrant",
    "C506": "Axillary extension (tail)",
    "C508": "Overlapping area of breast",
    "C509": "Breast, NOS"
}

morphology_map = {
    8000: "Neoplasm, NOS",
    8001: "Malignant tumor cells",
    8004: "Malignant tumor, spindle cell type",
    8010: "Carcinoma, NOS",
    8012: "Large cell carcinoma, NOS",
    8013: "Large cell neuroendocrine carcinoma",
    8020: "Undifferentiated carcinoma, NOS",
    8022: "Pleomorphic carcinoma",
    8030: "Giant cell and spindle cell carcinoma",
    8032: "Spindle cell carcinoma, NOS",
    8033: "Pseudosarcomatous carcinoma",
    8035: "Carcinoma with osteoclast-like giant cells",
    8041: "Small cell carcinoma, NOS",
    8045: "Mixed small and large cell carcinoma",
    8046: "Non-small cell carcinoma",
    8070: "Squamous cell carcinoma, NOS",
    8071: "Keratinizing squamous cell carcinoma",
    8074: "Spindle cell squamous carcinoma",
    8082: "Lymphoepithelial carcinoma",
    8140: "Adenocarcinoma, NOS",
    8141: "Scirrhous adenocarcinoma",
    8145: "Diffuse type adenocarcinoma",
    8200: "Adenoid cystic carcinoma",
    8201: "Cribriform carcinoma",
    8211: "Tubular adenocarcinoma",
    8230: "Solid carcinoma, NOS",
    8240: "Neuroendocrine tumor, NOS / Grade 1 (carcinoid)",
    8244: "Mixed adenoneuroendocrine carcinoma (MANEC)",
    8246: "Neuroendocrine carcinoma, NOS",
    8249: "Neuroendocrine tumor, grade 2/3 (atypical carcinoid)",
    8255: "Adenocarcinoma with mixed subtypes",
    8260: "Papillary adenocarcinoma, NOS",
    8290: "Oncocytic adenoma / carcinoma (Hurthle cell carcinoma)",
    8310: "Clear cell adenocarcinoma, NOS",
    8314: "Lipid-rich carcinoma",
    8315: "Glycogen-rich carcinoma",
    8401: "Apocrine adenocarcinoma",
    8407: "Microcystic adnexal carcinoma / Sclerosing sweat gland carcinoma",
    8410: "Sebaceous gland adenocarcinoma",
    8430: "Mucoepidermoid carcinoma",
    8441: "Serous cystadenocarcinoma, NOS",
    8470: "Mucinous cystadenocarcinoma, NOS",
    8480: "Mucinous adenocarcinoma",
    8481: "Mucin-producing adenocarcinoma",
    8490: "Signet ring cell carcinoma / 'poorly cohesive' carcinoma",
    8500: "Ductal carcinoma, NOS",
    8501: "Comedocarcinoma, NOS",
    8502: "Secretory carcinoma",
    8503: "Intraductal papillary adenocarcinoma",
    8504: "Encapsulated (intracystic) papillary carcinoma",
    8507: "Intraductal micropapillary carcinoma",
    8508: "Cystic hypersecretory carcinoma",
    8509: "Solid papillary carcinoma",
    8510: "Medullary carcinoma, NOS",
    8512: "Medullary carcinoma with lymphoid stroma",
    8513: "Atypical medullary carcinoma",
    8514: "Ductal carcinoma, desmoplastic type",
    8519: "Pleomorphic lobular carcinoma in situ",
    8520: "Lobular carcinoma, NOS",
    8521: "Ductular carcinoma",
    8522: "Ductal and lobular carcinoma",
    8523: "Ductal carcinoma mixed with another carcinoma type",
    8524: "Lobular carcinoma mixed with another carcinoma type",
    8530: "Inflammatory carcinoma",
    8540: "Paget's disease of the breast",
    8541: "Paget's disease + invasive ductal carcinoma",
    8543: "Paget's disease + intraductal carcinoma (DCIS)",
    8550: "Acinar cell carcinoma",
    8560: "Adenosquamous carcinoma",
    8562: "Epithelial-myoepithelial carcinoma",
    8570: "Adenocarcinoma with squamous metaplasia",
    8571: "Adenocarcinoma with cartilaginous or bony metaplasia",
    8572: "Adenocarcinoma with spindle cell metaplasia",
    8573: "Adenocarcinoma with apocrine metaplasia",
    8574: "Adenocarcinoma with neuroendocrine differentiation",
    8575: "Metaplastic carcinoma, NOS",
    8980: "Carcinosarcoma, NOS",
    8982: "Myoepithelial carcinoma",
    8983: "Malignant adenomyoepithelioma"
}

therapie_map = {
    100000: "Surgery, NOS",
    120000: "Local tumor resection",
    "130C50": "Breast-conserving surgery, NOS",
    "131C50": "Lumpectomy (without axillary lymph node dissection)",
    "132C50": "Lumpectomy (with axillary lymph node dissection)",
    "140C50": "Non-breast-conserving surgery, NOS",
    "141C50": "Mastectomy (without axillary lymph node dissection)",
    "142C50": "Mastectomy (with axillary lymph node dissection)",
    190000: "Resection for another indication (incidental finding)",
    315000: "Regional lymph node dissection for metastases",
    690100: "Surgical treatment abroad"
}

FEATURE_PROMPT_LABELS = {
    "leeft": "Age",
    "gesl": "Gender",
    "incjr": "IncidentYear",
    "tumsoort": "TumorType",
    "diag_basis": "DiagnosisBasis",
    "topo_sublok": "TopographySub",
    "later": "Laterality",
    "morf": "Morphology",
    "gedrag": "Behavior",
    "diffgrad": "DifferentiationGrade",
    "stadium": "Stage",
    "ond_lymf": "NumLymphExamined",
    "pos_lymf": "NumLymphPositive",
    "er_stat": "ERstatus",
    "pr_stat": "PRstatus",
    "her2_stat": "HER2status",
    "dcis_comp": "DCIScomponent",
    "multifoc": "Multifocality",
    "tum_afm": "TumorSize(mm)",
    "swk": "SentinelProcedure",
    "swk_uitslag": "SentinelOutcome",
    "okd": "AxillaryDissection",
    "org_chir": "OrganSurgery",
    "uitgebr_chir_code": "SurgeryCode",
    "dir_reconstr": "DirectReconstruction",
    "chemo": "Chemo",
    "target": "TargetTherapy",
    "horm": "HormoneTherapy",
    "rt": "Radiotherapy",
    "meta_rt": "RTMetastases",
    "meta_chir": "SurgeryMetastases"
}
FEATURES_TO_INCLUDE = list(FEATURE_PROMPT_LABELS.keys())

In [4]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from datasets import load_from_disk

# **Load T5-Base Model**
MODEL_NAME = "t5-base"
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, device_map="auto", torch_dtype="auto")

# **Load Tokenizer**
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME, legacy=False)

# **Load Dataset**
dataset = load_from_disk("./datasets/trainfold_0_dead")


def build_prompt(example):
    """
    Construct a descriptive string prompt from the row's data,
    mapping coded values to human-readable text.
    """
    gender_str = gender_map.get(example["gesl"], f"Unknown(gender={example['gesl']})")
    tumor_type_str = tumor_type_map.get(example["tumsoort"], f"Unknown(tumsoort={example['tumsoort']})")
    behavior_str = behavior_map.get(example["gedrag"], f"Unknown(gedrag={example['gedrag']})")
    diff_str = diffgrad_map.get(example["diffgrad"], f"Unknown(diff={example['diffgrad']})")
    later_key = str(example["later"])
    later_str = later_map.get(later_key, f"Unknown(later={example['later']})")
    topo_sub_str = topo_sublok_map.get(example["topo_sublok"], f"Unknown(topo_sublok={example['topo_sublok']})")
    morph_str = morphology_map.get(example["morf"], f"Unknown(morf={example['morf']})")
    er_str = hr_stat_map.get(example["er_stat"], f"Unknown(ER={example['er_stat']})")
    pr_str = hr_stat_map.get(example["pr_stat"], f"Unknown(PR={example['pr_stat']})")
    her2_str = her2_stat_map.get(example["her2_stat"], f"Unknown(HER2={example['her2_stat']})")

    prompt_lines = [
        f"Predict the survival time in days based on the following patient information:",
        f"Patient age: {int(example['leeft'])} years old.",
        f"Gender: {gender_str}.",
        f"Incident year: {int(example['incjr'])}.",
        f"Tumor type: {tumor_type_str}, morphology: {morph_str}, behavior: {behavior_str}.",
        f"Location: {later_str} breast, sublocation: {topo_sub_str}.",
        f"Differentiation grade: {diff_str}.",
        f"ER status: {er_str}, PR status: {pr_str}, HER2 status: {her2_str}."
    ]
    prompt_text = "\n".join(prompt_lines)
    prompt_text += "\nfSurvival time in days: "
    return prompt_text


def preprocess_function(example):
    """
    Build the prompt for an example, tokenize it, and attach the label.
    """
    prompt_text = build_prompt(example)
    tokenized = tokenizer(prompt_text, truncation=True, padding="max_length", max_length=256)
    tokenized_target = tokenizer(str(example["time_to_os"]), truncation=True, padding="max_length", max_length=5)
    tokenized["labels"] = tokenized_target["input_ids"]
    return tokenized


tokenized_dataset = dataset.map(preprocess_function, batched=False)
train_dataset = tokenized_dataset


# **Training Arguments**
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    weight_decay=0.01,
    fp16=True,
    logging_dir="./logs",
    logging_steps=1000,
    save_total_limit=3,
    # load_best_model_at_end=True,
    report_to = "none",
)

# **Trainer**
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    # eval_dataset=eval_dataset
)

# **Train the model**
trainer.train()

trainer.save_model("./t5_finetuned_dead")
tokenizer.save_pretrained("./t5_finetuned_dead")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Map:   0%|          | 0/9552 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
1000,2.063600
2000,1.656600
3000,1.625700
4000,1.625700
5000,1.614600
6000,1.613400
7000,1.606000
8000,1.602200
9000,1.600700
10000,1.595700


('./t5_finetuned_dead/tokenizer_config.json',
 './t5_finetuned_dead/special_tokens_map.json',
 './t5_finetuned_dead/spiece.model',
 './t5_finetuned_dead/added_tokens.json')

In [5]:
import torch
import json
from tqdm import tqdm

model.eval()

def predict_score(review_text):
    prompt = f"Review: {review_text} -> Score:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=10)

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the score (you may need extra parsing for clean output)
    predicted_score = generated_text.replace(prompt, "").strip()

    return predicted_score


test_dataset = load_from_disk("./datasets/testfold_0")
results = {"predicted": [], "real": [], "status": []}
for row in tqdm(test_dataset):
    text = build_prompt(row)
    predicted_score = predict_score(text)
    results["predicted"].append(predicted_score)
    results["real"].append(row["time_to_os"])
    results["status"].append(row["os"])
    # print(f"Predicted Score: {predicted_score}", f"Real score: {row['time_to_os']}", f"Status: {row['os']}")
    # print(row)
    # break

with open("results.json", "w") as f:
    json.dump(results, f)

100%|██████████| 12000/12000 [16:54<00:00, 11.82it/s]


In [6]:
!pip install lifelines

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 10.3 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=c30f7622029960a75dcd6bdbc1910d61504d1934ff6b963c20446c6604cefbd6
  Stored in directory: /root/.cache/pip/wheels/8b/67/f4/2caaae2146198dcb824f31a303833b07b14a5ec863fb3acd7b
Successfully built autograd-gamma


In [7]:
from lifelines.utils import concordance_index
import numpy as np

# Extract actual and predicted survival times
actual_times = np.array(results["real"])  # True survival times
predicted_times = np.array(results["predicted"])  # Model predictions
event_observed = np.array(results["status"])  # 1 if the event (death) occurred, 0 if censored

# Compute C-index considering censoring
c_index = concordance_index(actual_times, predicted_times, event_observed)
print(f"Concordance Index (C-Index) with Censoring: {c_index:.4f}")

Concordance Index (C-Index) with Censoring: 0.5567
